In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import collections
import gc
import pickle
import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Lambda, Flatten, Conv2DTranspose
from tensorflow.keras.layers import Dropout, GaussianNoise, Input, UpSampling2D
from tensorflow.keras.models import Model, Sequential, load_model
import sklearn.model_selection
from copy import deepcopy

<h2> Data Preprocessing </h2>

In [2]:
start = time.time()
data = pd.read_csv("./train.csv")
X1 = data[data.columns[np.vectorize(lambda s: "start" in s)(data.columns.values)]]
X1 = X1.values.reshape(data.shape[0],25,25,1)
delta = data["delta"].values
X2 = np.zeros((data.shape[0],25*25*5))
for i in range(data.shape[0]):
    X2[i,(delta[i]-1)::5] = 1
temp = data[data.columns[np.vectorize(lambda s: "stop" in s)(data.columns.values)]].values
Y = np.zeros((data.shape[0],25*25*5))
for i in range(data.shape[0]):
    Y[i,(delta[i]-1)::5] = temp[i]
print("Preprocessing Exited in "+str(time.time()-start)+" seconds")
data.shape, X1.shape, X2.shape, Y.shape

Preprocessing Exited in 7.875182867050171 seconds


((50000, 1252), (50000, 25, 25, 1), (50000, 3125), (50000, 3125))

<h2>Approach 1: N-Step Convolver</h2><br/>
We attempt to use a series of convolutions that maintain same padding to move from current Game of Life state to the outputs states. Specifically, we create a weighted_mean_squared_error custom loss function, wrapped inside a call-back and a class that decrements the higher weight to positive class over time.

In [22]:
class WeightedMSE:
    def __init__(self, origweight, targetweight, decayepoch):
        self.weight = origweight
        self.decay = (origweight-targetweight)/(decayepoch-1)
        self.endepoch = decayepoch
    def loss(self, ytrue, ypred):
        diff = ytrue-ypred
        poserr = tf.map_fn(lambda s: tf.reduce_sum(tf.math.square(s[s>0])), diff)*self.weight
        negerr = tf.map_fn(lambda s: tf.reduce_sum(tf.math.square(s[s<0])), diff)
        return tf.reduce_mean(poserr)+tf.reduce_mean(negerr)
    def update_weight(self, epoch):
        if epoch<self.endepoch:
            self.weight-=self.decay
        pickle.dump({}, open("./"+str(epoch)+".pkl","wb"))
class LossCallBack(keras.callbacks.Callback):
    def __init__(self, lossclass):
        self.lossclass = lossclass
    def on_epoch_end(self, epoch, logs):
        self.lossclass.update_weight(epoch)

In [13]:
numepochs, decayepochs = 7, 4
lossclass = WeightedMSE((625-np.sum(Y,axis=1).mean())/np.sum(Y,axis=1).mean(), 2, decayepochs)
losscallback = LossCallBack(lossclass)

In [14]:
tf.keras.backend.clear_session()
deltainp = Input((25*25*5), dtype=tf.float32) #Delta Mask
gofinp = Input((25,25,1)) #Game of Life Board
#Increasing Filters to Capture Relationships
conv1 = Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(gofinp)
max1 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="same")(conv1)
conv2 = Conv2D(filters=64, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(max1)
max2 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="same")(conv2)
#Reducing Filters to Final Output
conv3 = Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(max2)
max3 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="same")(conv3)
final = Conv2D(filters=5, kernel_size=(7,7), strides=(1,1), padding="same", activation=None)(max3)
#Evaluate Required Board
final = tf.keras.layers.Activation(activation="sigmoid")(Flatten()(final))
final = Flatten()(tf.math.multiply(final, deltainp))
model = Model(inputs=[gofinp, deltainp], outputs=final)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-3), loss=lossclass.loss)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 25, 25, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 25, 25, 32)   1600        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 25, 25, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 25, 25, 64)   100416      max_pooling2d[0][0]              
______________________________________________________________________________________________

In [15]:
hist = model.fit([X1[:40000],X2[:40000]],Y[:40000], batch_size=256, epochs=numepochs,
                 validation_data=([X1[40000:], X2[40000:]], Y[40000:]), callbacks=[losscallback])      
pd.Series(hist.history["loss"]).plot()
model.save("./40000example.h5")

Train on 40000 samples, validate on 10000 samples
Epoch 1/7
40000/40000 [==============================] - 246s 6ms/sample - loss: 174.9298 - val_loss: 152.4523
Epoch 2/7
40000/40000 [==============================] - 250s 6ms/sample - loss: 147.3668 - val_loss: 144.5295
Epoch 3/7
40000/40000 [==============================] - 250s 6ms/sample - loss: 141.8685 - val_loss: 140.8432
Epoch 4/7
11520/40000 [=======>......................] - ETA: 2:47 - loss: 139.2097

KeyboardInterrupt: 

In [21]:
losscallback.lossclass.weight

5.826105561517628

In [73]:
bounds = pd.Series(np.arange(0.5,0.9,0.04), index=np.arange(0.5,0.9,0.04))
for bound in bounds.index.values:
    totalcorr = 0
    for i in range(res.shape[0]):
        pred = deepcopy(res[i,(delta[40000+i]-1)::5])
        pred[pred<bound] = 0
        pred[pred>bound] = 1
        corr = Y[40000+i,(delta[40000+i]-1)::5]
        totalcorr+=sum(pred==corr)
    bounds[bound] = 1-totalcorr/(10000*625)

In [69]:
1-bounds/()

0.50    5187372.0
0.54    5187372.0
0.58    5187372.0
0.62    5187372.0
0.66    5187372.0
0.70    5187372.0
0.74    5187372.0
0.78    5187372.0
0.82    5187372.0
0.86    5187372.0
dtype: float64

<h2> Approach 2: CNN-Encoder-Decoder </h2>

In [153]:
tf.keras.backend.clear_session()
deltainp = Input((25,25,5), dtype=tf.float32) #Delta Mask
gofinp = Input((25,25,1)) #Game of Life Board
conv1 = Conv2D(filters=32, kernel_size=(5,5), strides=(1,1), padding="same", activation="relu")(gofinp)
max1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv1)
conv2 = Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), padding="same", activation="relu")(max1)
max2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv2)
drp2 = Dropout(0.1)(max2)
conv3 = Conv2D(filters=128, kernel_size=(5,5), strides=(1,1), padding="same", activation="elu")(drp2)
max3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv3)
up1 = UpSampling2D(size=(2,2), interpolation="nearest")(max3)
dec1 = Conv2DTranspose(filters=64, strides=(1,1), kernel_size=(5,5), padding="same", activation="elu")(up1)
up2 = UpSampling2D(size=(2,2), interpolation="nearest")(dec1)
dec2 = Conv2DTranspose(filters=32, strides=(1,1), kernel_size=(5,5), padding="same", activation="relu")(up2)
up3 = UpSampling2D(size=(2,2), interpolation="nearest")(dec2)
dec3 = Conv2DTranspose(filters=5, strides=(1,1), kernel_size=(2,2), activation="sigmoid")(up3)
final = Flatten()(tf.math.multiply(dec3, deltainp))
model = Model(inputs=[gofinp, deltainp], outputs=final)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss=weighted_mean_squared_error)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 25, 25, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 25, 25, 32)   832         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 12, 12, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 12, 12, 64)   51264       max_pooling2d[0][0]              
______________________________________________________________________________________________

<h2> Approach 3: LSTM-Based Sequence Generation